In [1]:
!pip install changeforest

In [2]:
import numpy as np
from changeforest import changeforest
from scipy.spatial.distance import cdist
from sklearn.metrics import adjusted_rand_score as sklearn_adjusted_rand_score
from tqdm import tqdm

In [3]:
dataset_generator = np.random.default_rng(12)

# Rand Index and Adjusted Rand index

The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.

let $tp,fp,tn$ and $fn$ be true positive pairs,false positive pairs,true negative pairs and false negative pairs.Then,

$$ RI = \frac{tp+tn}{tp+tn+fp+fn}$$

The adjusted rand index is rand index adjusted for chance 

$$ ARI = 2 \times \frac{tp \times tn - fn \times fp}{(tp + fn) \times (fn + tn) + (tp + fp) \times (fp + tn)}$$

Function to caculate the same

```
def calculate_rand_index_and_adjusted_rand_index(y_true,y_pred):
  pairs = []
  for i in range(len(y_true)):
    for j in range(i+1,len(y_true)):
      pairs.append((i,j))
  tp,tn,fp,fn =0,0,0,0
  for i,j in pairs:
    if y_true[i] == y_true[j] and y[j]==y[i]:
      tp+=1
    elif y_true[i] != y_true[j] and y[j]!=y[i]:
      tn+=1
    elif y_true[i]==y_true[j] and  y[j]!=y[i]:
      fn+=1
    elif y_true[i]!=y_true[j] and y[j]==y[i]:
      fp+=1
  rand_index = (tp+tn)/ (tp+tn+fp+fn)
  adjusted_rand_index = 2.0 * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) + (tp + fp) * (fp + tn))
  return rand_index,adjusted_rand_index

```

or you can use 
```
sklearn.metrics.adjusted_rand_score
```



In [5]:
def adjusted_rand_score(true_changepoints, estimated_changepoints):
    """Compute the adjusted rand index between two sets of changepoints.

    Args:
      true_changepoints (List) : list of true change points.
      estimated_changepoints (List) : list of estimated change points.
    """
    true_changepoints = np.array(true_changepoints, dtype=np.int_)
    estimated_changepoints = np.array(estimated_changepoints, dtype=np.int_)

    y_true = np.zeros(true_changepoints[-1])
    for i, (start, stop) in enumerate(zip(true_changepoints[:-1], true_changepoints[1:])):
        y_true[start:stop] = i

    y_estimated = np.zeros(estimated_changepoints[-1])
    for i, (start, stop) in enumerate(zip(estimated_changepoints[:-1], estimated_changepoints[1:])):
        y_estimated[start:stop] = i

    return sklearn_adjusted_rand_score(y_true, y_estimated)